# Install the dotnet agent

## Build

1. Clone the agent from repo.
2. Add it to the dotnet solution and project (SabberStonePython).
3. Build the solution.
4. Run

```shell
cd dotnet_core/ai
git clone https://github.com/rnilva/Hearthstone-MonteCarloGraphSearch

cd ../
dotnet sln add ai/Hearthstone-MonteCarloGraphSearch/MonteCarloGraphSearch/MonteCarloGraphSearch.csproj
dotnet add SabberStonePython.csproj reference ai/Hearthstone-MonteCarloGraphSearch/MonteCarloGraphSearch/MonteCarloGraphSearch.csproj

dotnet build -c Release -o ../python/_sabberstone_dotnet

dotnet ../python/_sabberstone_dotnet/SabberStonePython.dll
```

# Python API

## Structure of AbstractAI

```python
class AbstractAI(ABC):
    def __init__(self, stub, account_name):
        self._stub = stub
        self.metadata = (('id', account_name),)
        self.game_id = ""
        self.response_stream = None
        self.request_stream = None

    @abstractmethod
    def get_option(self, game: Game, sabber_stub) -> Option:
        """Calculate and reutnr the best move
        with regard to the given game state.
        Each AI agent must implement this method.
        """
        pass

    @abstractmethod
    def on_match_started(self):
        """This method will be called when a match is started."""
        pass

    @abstractmethod
    def on_game_started(self):
        """This method will be called when each game(round) is started."""
        pass

    @abstractmethod
    def on_game_finished(self, game):
        """This method will be called when each game(round) is finished."""
        pass
```


## Run SabberStone gRPC stub

In [1]:
import sys
sys.path.insert(0, "../../../")

In [5]:
import grpc
from pysabberstone.python.rpc import python_pb2_grpc

In [6]:
channel = grpc.insecure_channel("localhost:50052")
stub = python_pb2_grpc.SabberStonePythonStub(channel)
dotnet_ai_stub = python_pb2_grpc.DotnetAIServiceStub(channel)

## Create Python `AbstractAI` instance
In this tutorial we are going to use `RandomAI`.

In [7]:
from pysabberstone.python.ai.abstract_ai import RandomAI, dotnet_ai_match

In [8]:
account_name = "TestPythonRandomAI"
ai = RandomAI(stub, account_name)

## Run `dotnet_ai_match()`

In [14]:
deck = r"AAEBAQcCrwSRvAIOHLACkQP/A44FqAXUBaQG7gbnB+8HgrACiLACub8CAA=="

dotnet_ai_match(python_ai=ai,
                dotnet_ai_name="MonteCarloGraphSearch",
                python_ai_deckstring=deck,
                dotnet_ai_deckstring=deck,
                sabber_stub=stub,
                dotnet_ai_stub=dotnet_ai_stub,
                history=True)